<a href="https://colab.research.google.com/github/MHathorn/MSc-Dissertation/blob/main/Ramp_import.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install radiant_mlhub
!mlhub configure

In [ ]:
# Install needed Packages
%%capture
!pip install rasterio
!pip install geopandas
!pip install fastai --upgrade

In [ ]:
# Import packages
import rasterio
from rasterio.features import rasterize
import geopandas as gpd
from shapely.ops import cascaded_union
from shapely.ops import unary_union
from shapely.geometry import mapping, Point, Polygon, MultiPolygon
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import cv2
from fastai.vision.all import *
from tqdm import tqdm
from google.colab import drive

/usr/local/lib/python3.10/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [ ]:
drive.mount('/content/drive')
path = Path(f"/content/ramp_images_vm")


Mounted at /content/drive


In [ ]:
ramp_list = ['ramp_mzuzu_malawi', 'ramp_accra_ghana']

In [ ]:
from radiant_mlhub import Dataset

for i in ramp_list:
  ds = Dataset.fetch(i)
  ds.download(output_dir = path)
  for c in ds.collections:
      print(c.id)

ramp_mzuzu_malawi: fetch stac catalog: 1602KB [00:00, 26520.77KB/s]            
download assets: 100%|██████████| 6715/6715 [14:44<00:00,  7.59it/s]


ramp_mzuzu_malawi_labels
ramp_mzuzu_malawi_source


ramp_accra_ghana: fetch stac catalog: 786KB [00:00, 13883.92KB/s]            
download assets: 100%|██████████| 2661/2661 [05:41<00:00,  7.80it/s]


ramp_accra_ghana_labels
ramp_accra_ghana_source


In [ ]:
def tile_img_msk(image_path, tile_size:int):
  '''Split images and corresponding masks into smaller tiles and save them'''

  # Loop through all images in the given folder
  for fn in tqdm(image_path):
    output_path = fn.parent.parent

    # Create 'image_tiles' and 'mask_tiles' directories if they don't already exist
    if not os.path.exists(output_path/'image_tiles'):
      os.makedirs(output_path/'image_tiles')
    if not os.path.exists(output_path/f'{mask_type}_mask_tiles'):
      os.makedirs(output_path/f'{mask_type}_mask_tiles')

    # Create mask for current image
    img = np.array(PILImage.create(fn))
    msk_fn = str(fn).replace('images', f'untiled masks/{mask_type}')
    msk = np.array(PILMask.create(msk_fn))
    x, y, _ = img.shape

    # Cut tiles and save them
    for i in range(x//tile_size):
      for j in range(y//tile_size):
        img_tile = img[i*tile_size:(i+1)*tile_size,j*tile_size:(j+1)*tile_size]
        msk_tile = msk[i*tile_size:(i+1)*tile_size,j*tile_size:(j+1)*tile_size]
        Image.fromarray(img_tile).save(f'{output_path}/image_tiles/{fn.name[:-4]}_{i}_{j}.png')
        Image.fromarray(msk_tile).save(f'{output_path}/{mask_type}_mask_tiles/{fn.name[:-4]}_{i}_{j}.png')


def tile_img(image_path, output_path, tile_size:int, single = None):
  '''Tile image into smaller tiles and save them'''
  if single == False:
    for fn in tqdm(image_path):
      # Create output directory if it doesn't already exist
      if not os.path.exists(output_path):
        os.makedirs(output_path)

      # Create subdirectory for current image if it doesn't already exist
      sub_dir = f'{output_path}/{fn.name[:-4]}'
      if not os.path.exists(sub_dir):
        os.makedirs(sub_dir)

      # Create mask for current image
      img = np.array(PILImage.create(fn))
      x, y, _ = img.shape

      # Cut tiles and save them
      for i in range(x//tile_size):
        for j in range(y//tile_size):
          img_tile = img[i*tile_size:(i+1)*tile_size, j*tile_size:(j+1)*tile_size]
          Image.fromarray(img_tile).save(f'{sub_dir}/{fn.name[:-4]}_{i}_{j}.png')

  if single == True:
    # Create directories
    if not os.path.exists(output_path):
      os.makedirs(output_path)

    # Create mask for current image
    img = np.array(PILImage.create(image_path))
    x, y, _ = img.shape

    # Cut tiles and save them
    fn = image_path.split("/")[-1][:-4]
    for i in range(x//tile_size):
      for j in range(y//tile_size):
        img_tile = img[i*tile_size:(i+1)*tile_size,j*tile_size:(j+1)*tile_size]
        Image.fromarray(img_tile).save(f'{output_path}/{fn}/{fn}_{i}_{j}.png')


def generate_mask(raster_path, shape_path, output_path = None, file_name = None):
    '''Function that generates a binary mask from a vector file (shp or geojson)
    raster_path = path to the .tif;
    shape_path = path to the shapefile or GeoJson.
    output_path = Path to save the binary mask.
    file_name = Name of the file.'''

    # Load raster
    with rasterio.open(raster_path, "r") as src:
        raster_img = src.read()
        raster_meta = src.meta
        raster_crs = src.crs

    # Load shapefile
    train_df = gpd.read_file(shape_path)
    shape_crs = train_df.crs # get the CRS of the shapefile


    # Verify CRS of the raster file
    if train_df.crs != src.crs:
      print(f'Raster CRS: {src.crs}, Vector CRS: {train_df.crs}.\n Convert vector and raster to the same CRS.')
      # convert the shapefile to the raster CRS
      train_df = train_df.to_crs(raster_crs)
      # update the shape CRS
      shape_crs = train_df.crs

    # Generate the mask
    def poly_from_utm(polygon, transform):
        poly_pts = []
        poly = cascaded_union(polygon)
        for i in np.array(poly.exterior.coords):
            poly_pts.append( ~ transform * tuple(i))
        new_poly = Polygon(poly_pts)
        return new_poly

    poly_shp = []
    im_size = (src.meta['height'], src.meta['width'])
    for num, row in train_df.iterrows():
        if row['geometry'].geom_type == 'MultiPolygon':
            for p in row['geometry'].geoms: # iterate over polygons within a MultiPolygon
                poly = poly_from_utm(p, src.meta['transform'])
                poly_shp.append(poly)
        elif row['geometry'].geom_type == 'Polygon':
            poly = poly_from_utm(row['geometry'], src.meta['transform'])
            poly_shp.append(poly)
        else:
            # raise an error or skip the object
            raise TypeError("Invalid geometry type")
    # for num, row in train_df.iterrows():
    #     if row['geometry'].geom_type == 'Polygon':
    #         poly = poly_from_utm(row['geometry'], src.meta['transform'])
    #         poly_shp.append(poly)
    #     else:
    #         for p in row['geometry']:
    #             poly = poly_from_utm(p, src.meta['transform'])
    #             poly_shp.append(poly)



    if len(poly_shp) > 0:
        mask = rasterize(shapes=poly_shp, out_shape=im_size)
    else:
        mask = np.zeros(im_size)

    if erode:
      kernel = np.ones((3, 3), np.uint8)
      mask = cv2.erode(mask, kernel, iterations=1)

    # Save or show mask
    mask = mask.astype('uint8')
    bin_mask_meta = src.meta.copy()
    bin_mask_meta.update({'count': 1})
    if (output_path != None and file_name != None):
        os.chdir(output_path)
        with rasterio.open(file_name, 'w', **bin_mask_meta) as dst:
            dst.write(mask * 255, 1) # Change 255 to 1 if classes need to be 0 and 1
    else:
        return mask


def save_masks(images, mask, maskdir):
    if not os.path.exists(maskdir):
        os.makedirs(maskdir)
    for image in tqdm(images):
        if image.name.endswith(('.TIF', '.tif')):
            shapes = image.name
            generate_mask(image, mask, maskdir, shapes)


def informal_pixels_from_directory(tiled_masks_path):
    output_dict = {}
    for file in os.listdir(tiled_masks_path):
        try:
            mask = rasterio.open(os.path.join(tiled_masks_path, file))
            mask = (mask).read()
            settlement_pixels = np.count_nonzero(mask == 1)
            total_pixels = mask.size
            settlement_share = settlement_pixels / total_pixels
            output_dict[file] = settlement_share
        except:
            print(f'File {file} encountered an error.')
            continue
    return output_dict

def informal_pixels_from_file(file):
    output_dict = {}
    mask = rasterio.open(file)
    mask = mask.read()
    settlement_pixels = np.count_nonzero(mask == 1)
    total_pixels = mask.size
    settlement_share = settlement_pixels / total_pixels
    return settlement_share

In [ ]:
erode = True
image_dir = f'{path}/imagestest' # change this to your image directory
label_dir = f'{path}/labelstest' # change this to your label directory

# create the image and label directories if they do not exist
if not os.path.exists(image_dir):
  os.makedirs(image_dir, exist_ok=True)

if not os.path.exists(label_dir):
  os.makedirs(label_dir, exist_ok=True)

for i in ramp_list:
  raster_dir = f'{path}/{i}/{i}_source' # change this to your raster directory
  vector_dir = f'{path}/{i}/{i}_labels' # change this to your vector directory

  # copy and rename all the raster files to the image directory
  for root, dirs, files in os.walk(raster_dir):
      for file in files:
          if file.endswith((".tif", ".tiff")): # change this to match your raster files
              raster_path = os.path.join(root, file)
              folder_name = os.path.basename(root) # get the name of the folder that contains the raster file
              image_code = folder_name.partition("source")[-1] # join the substrings after the first "_" with "_"
              new_file_name = i + image_code + ".tif" # create a new file name with the folder name and extension
              new_file_path = os.path.join(image_dir, new_file_name) # create a new file path with the image directory and new file name
              if not os.path.exists(new_file_path):
                shutil.copy(raster_path, new_file_path) # copy the raster file to the new file path

  # copy and rename all the vector files to the label directory
  for root, dirs, files in os.walk(vector_dir):
      for file in files:
          if file.endswith(".geojson"): # change this to match your vector files
              vector_path = os.path.join(root, file)
              folder_name = os.path.basename(root) # get the name of the folder that contains the vector file
              image_code = folder_name.partition("labels")[-1] # join the substrings after the first "_" with "_"
              new_file_name = i + image_code + ".geojson" # create a new file name with the folder name and extension
              new_file_path = os.path.join(label_dir, new_file_name) # create a new file path with the label directory and new file name
              if not os.path.exists(new_file_path):
                shutil.copy(vector_path, new_file_path) # copy the vector file to the new file path

In [ ]:
len(os.listdir(image_dir))

4687

In [ ]:
len(os.listdir(label_dir))

4687

In [ ]:
def copy_img_msk(image_path):
  '''Split images and corresponding masks into smaller tiles and save them'''

  # Loop through all images in the given folder
  for fn in tqdm(image_path):
    output_path = fn.parent.parent

    # Create 'image_tiles' and 'mask_tiles' directories if they don't already exist
    if not os.path.exists(output_path/'image_tiles'):
      os.makedirs(output_path/'image_tiles')
    if not os.path.exists(output_path/'mask_tiles'):
      os.makedirs(output_path/'mask_tiles')

    # Create mask for current image
    img = np.array(PILImage.create(fn))
    msk_fn = str(fn).replace('imagestest', 'raster_masks_test')
    msk = np.array(PILMask.create(msk_fn))
    x, y, _ = img.shape

    # Check if the output files already exist
    img_out = f'{output_path}/image_tiles/{fn.name}.png'
    msk_out = f'{output_path}/mask_tiles/{fn.name}.png'

    # Save the output files only if they don't exist
    if not os.path.exists(img_out):
      Image.fromarray(img).save(img_out)
    if not os.path.exists(msk_out):
      Image.fromarray(msk).save(msk_out)

    # Image.fromarray(img).save(f'{output_path}/image_tiles/{fn.name}.png')
    # Image.fromarray(msk).save(f'{output_path}/mask_tiles/{fn.name}.png')

In [ ]:
output_path = f"{path}/raster_masks_test" # change this to your desired output path
if not os.path.exists(output_path):
  os.makedirs(output_path)

In [ ]:
for i in ramp_list:

  raster_dir = f'{path}/{i}/{i}_source' # change this to your raster directory
  vector_dir = f'{path}/{i}/{i}_labels' # change this to your vector directory

  for root, dirs, files in os.walk(raster_dir):
      for file in files:
          if file.endswith((".tif", ".tiff")): # change this to match your raster files
              raster_path = os.path.join(root, file)
              folder_name = os.path.basename(root) # get the name of the folder that contains the raster file
              image_code = folder_name.partition("source")[-1] # join the substrings after the first "_" with "_"
              shape_path = os.path.join(vector_dir, f"{i}_labels{image_code}", "vector_labels.geojson") # change this to match your geojson files
              file_name = i + image_code + ".tif"
              output_file = os.path.join(output_path, file_name)
              if not os.path.exists(output_file):
                print(f"Processing {file_name}")
                generate_mask(raster_path, shape_path, output_path, file_name)

Processing ramp_mzuzu_malawi_9a721772_b323_4297_80f4_19896f03d39f.tif


<ipython-input-9-a29656842bea>:98: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly = cascaded_union(polygon)
<ipython-input-9-a29656842bea>:98: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly = cascaded_union(polygon)
<ipython-input-9-a29656842bea>:98: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly = cascaded_union(polygon)


Processing ramp_mzuzu_malawi_68f2fe40_cabb_444b_8d7d_230994f9855c.tif
Processing ramp_mzuzu_malawi_2a6cac0d_a852_4a62_8c4f_7cb59cfc3521.tif
Processing ramp_mzuzu_malawi_343d98c9_35f5_40d9_b579_e9cc5cd61f4c.tif
Processing ramp_mzuzu_malawi_dc17729f_0708_49ed_913f_505207dc9144.tif
Processing ramp_mzuzu_malawi_8ca19a5b_d53d_4734_895b_1e7a7a49b6a7.tif
Processing ramp_mzuzu_malawi_c21e292f_407a_4a52_9fb5_63b4593f13ee.tif
Processing ramp_mzuzu_malawi_c246a55c_efa3_4227_a57d_9d746b1fe217.tif


<ipython-input-9-a29656842bea>:98: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly = cascaded_union(polygon)
<ipython-input-9-a29656842bea>:98: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly = cascaded_union(polygon)
<ipython-input-9-a29656842bea>:98: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly = cascaded_union(polygon)
<ipython-input-9-a29656842bea>:98: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly = cascaded_union(polygon)
<ipython-input-9-a29656842bea>:98: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly = cascaded_union(polygon)
<ipython-input-9-a29656842bea>:98: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly 

Processing ramp_mzuzu_malawi_2768c8e2_0840_45e9_9d4a_5ceaa8709472.tif
Processing ramp_mzuzu_malawi_76d8364c_3605_4de9_b692_ca8d82c76d53.tif
Processing ramp_mzuzu_malawi_a4fb870c_7c01_45ab_b654_9752924cc73b.tif
Processing ramp_mzuzu_malawi_69ec422c_c3bc_4095_a438_e1362144ea93.tif
Processing ramp_mzuzu_malawi_5bd67e5a_a33f_4fb7_a92c_2a228969c11b.tif
Processing ramp_mzuzu_malawi_bc181950_6e0a_48a4_9616_cfb194ab6c26.tif
Processing ramp_mzuzu_malawi_2549bbb2_bfed_4994_bbea_cb3c792c824f.tif


...

<ipython-input-9-a29656842bea>:98: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly = cascaded_union(polygon)
<ipython-input-9-a29656842bea>:98: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly = cascaded_union(polygon)
<ipython-input-9-a29656842bea>:98: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly = cascaded_union(polygon)
<ipython-input-9-a29656842bea>:98: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly = cascaded_union(polygon)


Raster CRS: EPSG:32630, Vector CRS: EPSG:4326.
 Convert vector and raster to the same CRS.
Processing ramp_accra_ghana_a9b0e8e8_54d7_4866_a01f_06d57f0e4197.tif
Raster CRS: EPSG:32630, Vector CRS: EPSG:4326.
 Convert vector and raster to the same CRS.
Processing ramp_accra_ghana_a7469dfd_8652_4b8a_a63a_4662ef5ebd74.tif
Raster CRS: EPSG:32630, Vector CRS: EPSG:4326.
 Convert vector and raster to the same CRS.
Processing ramp_accra_ghana_282f883a_b25b_4672_896f_ebdf5eaa1faa.tif
Raster CRS: EPSG:32630, Vector CRS: EPSG:4326.
 Convert vector and raster to the same CRS.
Processing ramp_accra_ghana_2414db2c_a1f8_43b8_bab8_bb607b526e9a.tif
Raster CRS: EPSG:32630, Vector CRS: EPSG:4326.
 Convert vector and raster to the same CRS.
Processing ramp_accra_ghana_40486a97_9880_4a28_aa66_aad1167842ae.tif


<ipython-input-9-a29656842bea>:98: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly = cascaded_union(polygon)
<ipython-input-9-a29656842bea>:98: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly = cascaded_union(polygon)
<ipython-input-9-a29656842bea>:98: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly = cascaded_union(polygon)
<ipython-input-9-a29656842bea>:98: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly = cascaded_union(polygon)


Raster CRS: EPSG:32630, Vector CRS: EPSG:4326.
 Convert vector and raster to the same CRS.
Processing ramp_accra_ghana_ecc735ee_1cdb_4281_9ee6_a92a9193acf4.tif
Raster CRS: EPSG:32630, Vector CRS: EPSG:4326.
 Convert vector and raster to the same CRS.
Processing ramp_accra_ghana_6d6e761e_1786_49e9_a6c6_68163473dfea.tif
Raster CRS: EPSG:32630, Vector CRS: EPSG:4326.
 Convert vector and raster to the same CRS.
Processing ramp_accra_ghana_c90b93b1_84d4_4f57_842c_1d650e9bfa0b.tif
Raster CRS: EPSG:32630, Vector CRS: EPSG:4326.
 Convert vector and raster to the same CRS.
Processing ramp_accra_ghana_31057233_65e4_4584_aae2_039e210b6b0e.tif
Raster CRS: EPSG:32630, Vector CRS: EPSG:4326.
 Convert vector and raster to the same CRS.


<ipython-input-9-a29656842bea>:98: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly = cascaded_union(polygon)
<ipython-input-9-a29656842bea>:98: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly = cascaded_union(polygon)
<ipython-input-9-a29656842bea>:98: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  poly = cascaded_union(polygon)


In [ ]:
images_list = get_image_files(image_dir)
copy_img_msk(images_list)

100%|██████████| 4687/4687 [02:53<00:00, 27.07it/s]


In [ ]:
from IPython.lib.display import exists
dest_dir = "/content/drive/MyDrive/BEAM/training_imagery"
if not os.path.exists(dest_dir):
  os.makedirs(dest_dir)

images = f'{path}/image_tiles'
masks = f'{path}/mask_tiles'

# Use the shell command to zip the folders
!zip -r images_masks.zip {images} {masks}

# Copy the zip file to the destination folder
shutil.copy("images_masks.zip", dest_dir)

Streaming output truncated to the last 5000 lines.
updating: content/ramp_images_vm/image_tiles/ramp_accra_ghana_f20bcf96_b6ff_43f1_a785_fc136494bcf5.tif.png (deflated 0%)
updating: content/ramp_images_vm/image_tiles/ramp_mzuzu_malawi_bc181950_6e0a_48a4_9616_cfb194ab6c26.tif.png (deflated 0%)
updating: content/ramp_images_vm/image_tiles/ramp_mzuzu_malawi_6ccb19d9_f40e_4362_b407_e507f7125a64.tif.png (deflated 0%)
updating: content/ramp_images_vm/image_tiles/ramp_mzuzu_malawi_2d8f9717_2bef_4374_bd50_78aa16886b1e.tif.png (deflated 0%)
updating: content/ramp_images_vm/image_tiles/ramp_mzuzu_malawi_abfbb77e_ff70_40b4_ab5d_30cdb3fdc52d.tif.png (deflated 0%)
updating: content/ramp_images_vm/image_tiles/ramp_mzuzu_malawi_11c94d09_d554_4989_9c16_57c25ee7719f.tif.png (deflated 0%)
updating: content/ramp_images_vm/image_tiles/ramp_accra_ghana_c8e897a0_2d20_4fa2_9c64_583c3e06549b.tif.png (deflated 0%)
updating: content/ramp_images_vm/image_tiles/ramp_mzuzu_malawi_fc92d419_0df6_4d3a_b851_42a8285256

'/content/drive/MyDrive/BEAM/training_imagery/images_masks.zip'

In [ ]:
from IPython.lib.display import exists
dest_dir = "/content/drive/MyDrive/BEAM/training_imagery"
if not os.path.exists(dest_dir):
  os.makedirs(dest_dir)

images = f'{path}/image_tiles'
masks = f'{path}/mask_tiles'

# Use the shell command to zip the folders
!zip -r ramp_download.zip {path}

# Copy the zip file to the destination folder
shutil.copy("ramp_download.zip", dest_dir)

Streaming output truncated to the last 5000 lines.
  adding: content/ramp_images_vm/raster_masks_test/ramp_barishal_bangladesh_d5e0028d_d7c8_4364_9cb5_9a26a98c90a2.tif (deflated 99%)
  adding: content/ramp_images_vm/raster_masks_test/ramp_barishal_bangladesh_9244c60e_340c_4825_994b_79e01398b97b.tif (deflated 99%)
  adding: content/ramp_images_vm/raster_masks_test/ramp_mzuzu_malawi_0fcc762e_bd64_4ad6_a398_2dd55c53f688.tif (deflated 96%)
  adding: content/ramp_images_vm/raster_masks_test/ramp_muscat_oman_3de07deb_f158_4464_8f93_db6b491c8e2a.tif (deflated 97%)
  adding: content/ramp_images_vm/raster_masks_test/ramp_muscat_oman_8be30f19_a7fd_4453_b44c_fb57541c9b2a.tif (deflated 99%)
  adding: content/ramp_images_vm/raster_masks_test/ramp_muscat_oman_f922e427_d31f_4c28_a3f4_5033f05b9fe6.tif (deflated 95%)
  adding: content/ramp_images_vm/raster_masks_test/ramp_muscat_oman_a3705018_394b_43e9_8e8c_6fe3c08bc7d7.tif (deflated 98%)
  adding: content/ramp_images_vm/raster_masks_test/ramp_mzuzu_ma

'/content/drive/MyDrive/BEAM/ramp_images_vm/ramp_download.zip'